In [5]:
import time
import torch
import numpy as np
from torch.optim import Adam
from options.TrainOptions import *
from Encoder import *
from Decoder import *
from util import *

# def training(opt, encoder, decoder, train_loader, val_loader):
#     encoder.train()
#     decoder.train()

#     for epoch in range(opt.n_epoch):
#         avg_loss = 0
#         start = time.time()
#         for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(train_loader):

#             utterances = utterances.to(opt.device)
#             labels = labels.to(opt.device)
#             u_lens = u_lens.to(opt.device)
#             l_lens = l_lens.to(opt.device)

#             encoder.zero_grad()
#             decoder.zero_grad()
#             optimizer.zero_grad()

#             keys, values, out_lens = encoder(utterances, u_lens)
#             predict_labels = decoder(keys, values, labels)

#             loss = criterion(predict_labels, labels)
#             mask = torch.arange(labels.size(1)).unsqueeze(0) >= l_lens.unsqueeze(1)

#             loss.backward()
        # 	# `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        # 	nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        # 	avg_loss += loss.item()
        # 	if batch_idx % 10 == 9:
        # 		print('batch', batch_idx + 1, 'Loss', avg_loss / 10, 'Running time', time.time() - start)
        # 		avg_loss = 0

        # 	optimizer.step()

        # torch.save(model.state_dict(), './model_{}.pt'.format(epoch))
        # validation(model, val_loader)


opt = TrainOptions().parser.parse_args(args = [])
speech_train = np.load(opt.dataroot + 'train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load(opt.dataroot + 'dev_new.npy', allow_pickle=True, encoding='bytes')
# speech_test = np.load(opt.dataroot + 'test.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load(opt.dataroot + 'train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")
encoder = Encoder(opt)
decoder = Decoder(opt)
encoder.to(opt.device)
decoder.to(opt.device)
optimizer = Adam(list(encoder.parameters()) + list(decoder.parameters()), opt.lr)
criterion = nn.CrossEntropyLoss(reduction = 'none')
criterion.to(opt.device)

transcript_train = transform_letter_to_index(transcript_train)
transcript_valid = transform_letter_to_index(transcript_valid)

print("Transfer the transcript from letters to index sucessfully.....")
train_data = MyDataset(speech_train, transcript_train)
dev_data = MyDataset(speech_valid, transcript_valid)

train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)
dev_loader_args = dict(shuffle=False, batch_size = opt.val_batch_size, pin_memory=True, collate_fn = collate_fn) 
dev_loader = Data.DataLoader(dev_data, **dev_loader_args)

# training(opt, encoder, decoder, train_loader, dev_loader)



Data Loading Sucessful.....
Transfer the transcript from letters to index sucessfully.....


In [6]:
import os

In [12]:
os.path.isdir(opt.model_name)

True

In [10]:
os.exist(opt.model_name)

AttributeError: module 'os' has no attribute 'exist'

In [9]:
torch.save(encoder.state_dict(), './{}/encoder_{}.pt'.format(opt.model_name, 0))

In [4]:
encoder.train()
decoder.train()

# for epoch in range(opt.n_epoch):
#     avg_loss = 0
#     start = time.time()
for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(train_loader):
    utterances = utterances.permute(1, 0, 2)
    utterances = utterances.to(opt.device)
    labels = labels.to(opt.device)
    u_lens = u_lens.to(opt.device)
    l_lens = l_lens.to(opt.device)

    encoder.zero_grad()
    decoder.zero_grad()
    optimizer.zero_grad()

    keys, values, out_lens = encoder(utterances, u_lens)

    keys = keys.permute(1, 0, 2)
    values = values.permute(1, 0, 2)
    predict_labels = decoder(keys, values, labels, out_lens).permute(0, 2, 1)
    loss = criterion(predict_labels, labels)
    mask = torch.arange(labels.size(1)).unsqueeze(0).to(opt.device) >= l_lens.unsqueeze(1)
    loss.masked_fill_(mask, 0.0)
    loss = loss.sum() / l_lens.sum()
    loss.backward()
    break

torch.Size([640, 64, 512])
torch.Size([320, 64, 512])
torch.Size([160, 64, 512])


In [3]:
criterion = nn.CrossEntropyLoss()
criterion.cuda()

CrossEntropyLoss()

In [4]:
predict_labels.shape

torch.Size([64, 34, 207])

In [5]:
loss = criterion(predict_labels, labels)

In [6]:
loss.backward()

In [5]:
optimizer.zero_grad()

In [ ]:
loss.backward()

In [ ]:
loss.mean().backward()

In [3]:
loss = criterion(predict_labels, labels)
mask = torch.arange(labels.size(1)).unsqueeze(0) >= l_lens.unsqueeze(1)
loss.masked_fill_(mask, 0.0)

tensor([[3.5070, 3.5262, 3.5014,  ..., 0.0000, 0.0000, 0.0000],
        [3.5072, 3.5435, 3.5799,  ..., 0.0000, 0.0000, 0.0000],
        [3.5068, 3.4745, 3.5048,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [3.5064, 3.5276, 3.5533,  ..., 0.0000, 0.0000, 0.0000],
        [3.5068, 3.5393, 3.5069,  ..., 0.0000, 0.0000, 0.0000],
        [3.5063, 3.5448, 3.4923,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<AsStridedBackward>)

In [ ]:
loss = loss.mean()
loss.backward()

In [5]:
loss = pack_padded_sequence(loss, lengths = l_lens, batch_first = False, enforce_sorted = False).data

In [10]:
loss = loss.mean()

In [11]:
loss.backward()

IndexError: select(): index 64 out of range for tensor of size [64, 64] at dimension 0

In [3]:
keys = keys.permute(1, 0, 2)
values = values.permute(1, 0, 2)

In [4]:
predict_labels = decoder(keys, values, labels, out_lens)

In [8]:
labels.shape

torch.Size([64, 202])

torch.Size([64, 202, 34])

In [10]:
loss = criterion(predict_labels.permute(0, 2, 1), labels)

In [12]:
mask = torch.arange(labels.size(1)).unsqueeze(0) >= l_lens.unsqueeze(1)

In [13]:
loss.masked_fill_(mask, 0)

tensor([[3.5451, 3.4696, 3.5204,  ..., 0.0000, 0.0000, 0.0000],
        [3.5447, 3.5426, 3.5435,  ..., 0.0000, 0.0000, 0.0000],
        [3.5449, 3.5448, 3.5682,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [3.5453, 3.4648, 3.4188,  ..., 0.0000, 0.0000, 0.0000],
        [3.5455, 3.4259, 3.5422,  ..., 0.0000, 0.0000, 0.0000],
        [3.5452, 3.4263, 3.5423,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<AsStridedBackward>)

In [22]:
loss.sum()

tensor(23232.4785, grad_fn=<SumBackward0>)

In [24]:
l_lens.sum()

tensor(6570)

In [25]:
loss = loss.sum() / l_lens.sum()

In [ ]:
loss.backward()

In [19]:
torch.exp(loss)

tensor([[34.6444, 32.1235, 33.7985,  ...,  1.0000,  1.0000,  1.0000],
        [34.6292, 34.5567, 34.5867,  ...,  1.0000,  1.0000,  1.0000],
        [34.6375, 34.6340, 35.4520,  ...,  1.0000,  1.0000,  1.0000],
        ...,
        [34.6493, 31.9692, 30.5341,  ...,  1.0000,  1.0000,  1.0000],
        [34.6560, 30.7496, 34.5440,  ...,  1.0000,  1.0000,  1.0000],
        [34.6469, 30.7625, 34.5459,  ...,  1.0000,  1.0000,  1.0000]],
       grad_fn=<ExpBackward>)

In [13]:
a = torch.ones((64, 171, 128))
b = torch.ones((64, 128, 1))

In [14]:
torch.bmm(a, b).shape

torch.Size([64, 171, 1])

In [1]:
import time
import torch
import numpy as np
from torch.optim import Adam
from options.TrainOptions import *
from Encoder import *
from Decoder import *
from util import *

opt = TrainOptions().parser.parse_args(args = [])

speech_train = np.load(opt.dataroot + 'train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load(opt.dataroot + 'dev_new.npy', allow_pickle=True, encoding='bytes')
# speech_test = np.load(opt.dataroot + 'test.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load(opt.dataroot + 'train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")
encoder = Encoder(opt)
decoder = Decoder(opt)
encoder.to(opt.device)
decoder.to(opt.device)
optimizer = Adam(list(encoder.parameters()) + list(decoder.parameters()), opt.lr)
criterion = nn.CrossEntropyLoss(reduction = 'none')
criterion.to(opt.device)

transcript_train = transform_letter_to_index(transcript_train)
transcript_valid = transform_letter_to_index(transcript_valid)

print("Transfer the transcript from letters to index sucessfully.....")
train_data = MyDataset(speech_train, transcript_train)
dev_data = MyDataset(speech_valid, transcript_valid)

train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)
dev_loader_args = dict(shuffle=False, batch_size = opt.val_batch_size, pin_memory=True, collate_fn = collate_fn) 
dev_loader = Data.DataLoader(dev_data, **dev_loader_args)

----------------- Options ---------------
                 dataroot: ./data/                       
       decoder_hidden_dim: 512                           
       decoder_num_layers: 2                             
                   device: cuda                          
             display_freq: 10                            
           embedding_size: 256                           
       encoder_hidden_dim: 256                           
       encoder_num_layers: 4                             
                input_dim: 40                            
         is_bidirectional: True                          
                 key_size: 128                           
                       lr: 0.001                         
               model_name: LAS_latest                    
                  n_epoch: 10                            
         save_latest_freq: 1                             
    teacher_forcing_ratio: 0.9                           
         train_batch_size: 64 

In [2]:
encoder.load_state_dict(torch.load('./LAS_latest/encoder_0.pt'))

<All keys matched successfully>

In [3]:
decoder.load_state_dict(torch.load('./LAS_latest/decoder_0.pt'))

<All keys matched successfully>

In [6]:
def validation(opt, encoder, decoder, val_loader):
    encoder.eval()
    decoder.eval()

    start = time.time()
    running_loss = 0
    for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(val_loader):
        utterances = utterances.permute(1, 0, 2)
        utterances = utterances.to(opt.device)
        labels = labels.to(opt.device)
        u_lens = u_lens.to(opt.device)
        l_lens = l_lens.to(opt.device)
        keys, values, out_lens = encoder(utterances, u_lens)

        keys = keys.permute(1, 0, 2)
        values = values.permute(1, 0, 2)
        predict_labels = decoder(keys, values, labels, out_lens, mode = 'val').permute(0, 2, 1)
        loss = criterion(predict_labels, labels)
        mask = torch.arange(labels.size(1)).unsqueeze(0).to(opt.device) >= l_lens.unsqueeze(1)
        loss.masked_fill_(mask, 0.0)
        loss = loss.sum() / l_lens.sum()
        running_loss += torch.exp(loss).item()
    
    file_name = os.path.join('./' + opt.model_name, '{}.txt'.format(opt.model_name))
    with open(file_name, 'a') as opt_file:
        opt_file.write('='*16)
        opt_file.write('\n')
        opt_file.write('Perplexity = {}, Running time = {}'.format(running_loss / batch_idx, time.time() - start))
        opt_file.write('\n')

In [7]:
validation(opt, encoder, decoder, train_loader)

RuntimeError: invalid argument 7: equal number of batches expected at /opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/THC/generic/THCTensorMathBlas.cu:493

In [10]:
%debug

> /home/ubuntu/src/Homework-4-Part-2-11-785-Fall-2019/Atten.py(23)forward()
     21         query = self.linear(query)
     22         # Input/output shape of bmm: (N, T, key_size), (N, key_size, 1) -> (N, T, 1)
---> 23         attention = torch.bmm(key, query.unsqueeze(2)).squeeze(2)
     24 
     25         # Create an (N, T) boolean mask for all padding positions

ipdb> up
> /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/module.py(547)__call__()
    545             result = self._slow_forward(*input, **kwargs)
    546         else:
--> 547             result = self.forward(*input, **kwargs)
    548         for hook in self._forward_hooks.values():
    549             hook_result = hook(self, input, result)

ipdb> up
> /home/ubuntu/src/Homework-4-Part-2-11-785-Fall-2019/Decoder.py(56)forward()
     54                 char_embed = self.embedding(prediction.argmax(dim = 1))
     55 
---> 56             context, atten = self.attention(key, char_embe